In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ['XLA_PYTHON_CLIENT_PREALLOCATE'] = 'false'

from pathlib import Path
root_path = Path.cwd().parent.parent.parent.absolute()
import sys
current_dir = Path.cwd()
repo_root = current_dir.parent.parent.parent  # Go up 3 levels
pdpo_path = str(repo_root.absolute())
# Add repository root to path
sys.path.insert(0, pdpo_path)




In [1]:
import jax
import jax.numpy as jnp
import jax.random as jrn
from jax import jit, vmap
from flax import nnx
import optax
import numpy as np
import matplotlib.pyplot as plt
import seaborn
from typing import Dict, Tuple, Optional
import time

In [2]:
# Set plot style
plt.style.use('default')

# JAX Configuration
print("JAX version:", jax.__version__)
print("Available devices:", jax.devices())

device = jax.devices()[0]
jax.default_device(device)




JAX version: 0.7.0
Available devices: [CudaDevice(id=0), CudaDevice(id=1), CudaDevice(id=2), CudaDevice(id=3)]


In [ ]:
# Set random seed for reproducibility
SEED = 1
key = jrn.PRNGKey(SEED)
print(f"Random seed set to: {SEED}")